## BUILD A MODEL ON MNIST DATA

In [1]:
import tempfile
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [21]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0


model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Conv2D(filters=24, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Conv2D(filters=48, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1688/1688 [==============================] - 8s 4ms/step - loss: 0.3024 - accuracy: 0.9083 - val_loss: 0.1131 - val_accuracy: 0.9662
Epoch 2/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1135 - accuracy: 0.9652 - val_loss: 0.0818 - val_accuracy: 0.9767
Epoch 3/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0870 - accuracy: 0.9731 - val_loss: 0.0894 - val_accuracy: 0.9752
Epoch 4/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0708 - accuracy: 0.9779 - val_loss: 0.0682 - val_accuracy: 0.9785


In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 13, 13, 12)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 24)        2616      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 5, 5, 24)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 48)         

In [8]:
# pip install tensorflow_model_optimization

In [15]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch_size = 128
epochs = 2
validation_split = 0.1

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)
# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 28, 28, 1)         1         
 e (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_conv2d  (None, 26, 26, 12)        230       
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_max_po  (None, 13, 13, 12)        1         
 oling2d (PruneLowMagnitude                                      
 )                                                               
                                                                 
 prune_low_magnitude_conv2d  (None, 11, 11, 24)        5210      
 _1 (PruneLowMagnitude)                                          
                                                        

In [16]:
import tempfile
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 10s 7ms/step - loss: 0.1145 - accuracy: 0.9667 - val_loss: 0.1306 - val_accuracy: 0.9718
Epoch 2/2
422/422 [==============================] - 3s 6ms/step - loss: 0.1183 - accuracy: 0.9673 - val_loss: 0.0720 - val_accuracy: 0.9792


In [17]:
_, baseline_model_accuracy = model.evaluate(
   test_images, test_labels, verbose=0)
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9740999937057495
Pruned test accuracy: 0.9740999937057495


In [18]:
model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 28, 28, 1)         1         
 e (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_conv2d  (None, 26, 26, 12)        230       
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_max_po  (None, 13, 13, 12)        1         
 oling2d (PruneLowMagnitude                                      
 )                                                               
                                                                 
 prune_low_magnitude_conv2d  (None, 11, 11, 24)        5210      
 _1 (PruneLowMagnitude)                                          
                                                        

In [20]:
model.summmary()

AttributeError: 'Sequential' object has no attribute 'summmary'

In [10]:
pruned_model = tfmot.sparsity.keras.strip_pruning(
    model_for_pruning
)

In [11]:
# TRAIN THE DIGIT CLASSIFICATION MODEL
pruned_model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
pruned_model.fit(
  train_images,
  train_labels,
  epochs=5,
  validation_data=(test_images, test_labels)
)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0592 - accuracy: 0.9813 - val_loss: 0.0636 - val_accuracy: 0.9809
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0491 - accuracy: 0.9845 - val_loss: 0.0626 - val_accuracy: 0.9816
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0435 - accuracy: 0.9864 - val_loss: 0.0552 - val_accuracy: 0.9834
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0387 - accuracy: 0.9869 - val_loss: 0.0553 - val_accuracy: 0.9837
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0344 - accuracy: 0.9890 - val_loss: 0.0491 - val_accuracy: 0.9864


In [12]:
pruned_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 12)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 24)        2616      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 24)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 48)          1

In [13]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.0491 - accuracy: 0.9864


[0.04914211854338646, 0.9864000082015991]

In [14]:
pruned_model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.0491 - accuracy: 0.9864


[0.04914211854338646, 0.9864000082015991]